In [5]:
!pip install xlrd

  Using cached xlrd-1.2.0-py2.py3-none-any.whl (103 kB)


You should consider upgrading via the 'c:\python37\python.exe -m pip install --upgrade pip' command.


In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, Row
import pandas as pd

from pyspark.sql.functions import when, udf, col, regexp_replace
from pyspark.sql.types import DoubleType,IntegerType, StringType
import pyspark.sql.functions as func

import matplotlib.pyplot as plt

In [2]:
sc = SparkContext('local')
sqlCtx = SQLContext( sc ) 

In [6]:
pdf = pd.read_excel( 'data/hotel.xlsx')
pdf

,Unnamed: 0,grade,rate,price
0,호텔1,10,1,9500
1,호텔2,8,2,8000
2,호텔3,6,4,6000
3,호텔4,9,2,8600
4,호텔5,7,4,7000
5,호텔6,8,2,8300
6,호텔7,5,5,5000
7,호텔8,5,5,5200
8,호텔9,8,3,8100
9,호텔10,10,1,9300


In [7]:
df = sqlCtx.createDataFrame( pdf )
df.show()

+----------+-----+----+-----+
|Unnamed: 0|grade|rate|price|
+----------+-----+----+-----+
|     호텔1|   10|   1| 9500|
|     호텔2|    8|   2| 8000|
|     호텔3|    6|   4| 6000|
|     호텔4|    9|   2| 8600|
|     호텔5|    7|   4| 7000|
|     호텔6|    8|   2| 8300|
|     호텔7|    5|   5| 5000|
|     호텔8|    5|   5| 5200|
|     호텔9|    8|   3| 8100|
|    호텔10|   10|   1| 9300|
|    호텔11|    9|   3| 8800|
|    호텔12|    7|   4| 7100|
|    호텔13|    8|   3| 8500|
|    호텔14|    5|   4| 5200|
|    호텔15|    5|   5| 3200|
|    호텔16|    6|   4| 6800|
|    호텔17|    7|   4| 7100|
|    호텔18|    8|   3| 8400|
|    호텔19|    5|   4| 2200|
|    호텔20|    5|   5| 4500|
+----------+-----+----+-----+



In [8]:
df = df.withColumnRenamed('Unnamed: 0','hotelname')
df.show()

+---------+-----+----+-----+
|hotelname|grade|rate|price|
+---------+-----+----+-----+
|    호텔1|   10|   1| 9500|
|    호텔2|    8|   2| 8000|
|    호텔3|    6|   4| 6000|
|    호텔4|    9|   2| 8600|
|    호텔5|    7|   4| 7000|
|    호텔6|    8|   2| 8300|
|    호텔7|    5|   5| 5000|
|    호텔8|    5|   5| 5200|
|    호텔9|    8|   3| 8100|
|   호텔10|   10|   1| 9300|
|   호텔11|    9|   3| 8800|
|   호텔12|    7|   4| 7100|
|   호텔13|    8|   3| 8500|
|   호텔14|    5|   4| 5200|
|   호텔15|    5|   5| 3200|
|   호텔16|    6|   4| 6800|
|   호텔17|    7|   4| 7100|
|   호텔18|    8|   3| 8400|
|   호텔19|    5|   4| 2200|
|   호텔20|    5|   5| 4500|
+---------+-----+----+-----+



In [12]:
#sum, count, min, max, avg, agg
df.groupBy('grade').sum().show()

+-----+----------+---------+----------+
|grade|sum(grade)|sum(rate)|sum(price)|
+-----+----------+---------+----------+
|    7|        21|       12|     21200|
|    6|        12|        8|     12800|
|    9|        18|        5|     17400|
|    5|        30|       28|     25300|
|   10|        20|        2|     18800|
|    8|        40|       13|     41300|
+-----+----------+---------+----------+



In [15]:
df.groupBy('grade').sum('rate','price').show()

+-----+---------+----------+
|grade|sum(rate)|sum(price)|
+-----+---------+----------+
|    7|       12|     21200|
|    6|        8|     12800|
|    9|        5|     17400|
|    5|       28|     25300|
|   10|        2|     18800|
|    8|       13|     41300|
+-----+---------+----------+



In [16]:
df.groupBy('grade').avg('price').show()

+-----+-----------------+
|grade|       avg(price)|
+-----+-----------------+
|    7|7066.666666666667|
|    6|           6400.0|
|    9|           8700.0|
|    5|4216.666666666667|
|   10|           9400.0|
|    8|           8260.0|
+-----+-----------------+



In [17]:
df.groupBy('grade').max('price').show()

+-----+----------+
|grade|max(price)|
+-----+----------+
|    7|      7100|
|    6|      6800|
|    9|      8800|
|    5|      5200|
|   10|      9500|
|    8|      8500|
+-----+----------+



In [20]:
df.groupBy('grade').min('price').orderBy('grade').show()

+-----+----------+
|grade|min(price)|
+-----+----------+
|    5|      2200|
|    6|      6000|
|    7|      7000|
|    8|      8000|
|    9|      8600|
|   10|      9300|
+-----+----------+



In [22]:
df.groupBy('grade').agg( func.sum('price').alias('priceSum'), 
                        func.max('price').alias('priceMax') ).show()

+-----+--------+--------+
|grade|priceSum|priceMax|
+-----+--------+--------+
|    7|   21200|    7100|
|    6|   12800|    6800|
|    9|   17400|    8800|
|    5|   25300|    5200|
|   10|   18800|    9500|
|    8|   41300|    8500|
+-----+--------+--------+

